In [ ]:
!pip uninstall -y google-cloud-datastore

In [ ]:
!pip install google-cloud-datastore

In [ ]:
!pip install inflect

In [ ]:
# Only need to do this once...
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
from nltk.corpus import stopwords
stop = set(stopwords.words('french'))

In [ ]:
from google.cloud import datastore

In [ ]:
datastore_client = datastore.Client()

In [ ]:
client = datastore.Client()
query = client.query(kind='Topic')
results = list(query.fetch())

In [ ]:
import inflect
plurals = inflect.engine()

In [ ]:
from nltk.corpus import wordnet

for result in results:
    for word in result.key.name.split():
        if word in stop:
            continue
        synonyms = set()
        for syn in wordnet.synsets(word):
            if ".n." in str(syn):
                for l in syn.lemmas():
                    lemma = l.name()
                    if (lemma.isalpha()):
                        synonyms.add(lemma)
                        synonyms.add(plurals.plural(lemma))

            if ".a." in str(syn):
                synonyms = set()
                break

        print(result.key.name, word, synonyms)

        kind = 'Synonym'
        synonym_key = datastore_client.key(kind, result.key.name)

        synonym = datastore.Entity(key=synonym_key)
        synonym['synonym'] = result.key.name

        datastore_client.put(synonym)

        synonym_key = datastore_client.key(kind, word)

        synonym = datastore.Entity(key=synonym_key)
        synonym['synonym'] = result.key.name

        datastore_client.put(synonym)

        for dictionary_synonym in synonyms:
            synonym_key = datastore_client.key(kind, dictionary_synonym)
            synonym = datastore.Entity(key=synonym_key)
            synonym['synonym'] = result.key.name
            datastore_client.put(synonym)

        synonym_key = datastore_client.key(kind, plurals.plural(word))

        synonym = datastore.Entity(key=synonym_key)
        synonym['synonym'] = result.key.name

        datastore_client.put(synonym)